<a href="https://colab.research.google.com/github/lynn-e/ml-guide/blob/master/ch09/9_8_%ED%8C%8C%EC%9D%B4%EC%8D%AC_%EC%B6%94%EC%B2%9C_%EC%8B%9C%EC%8A%A4%ED%85%9C_%ED%8C%A8%ED%82%A4%EC%A7%80_Surprise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install surprise

     |████████████████████████████████| 6.5MB 2.7MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.0-cp36-cp36m-linux_x86_64.whl size=1675743 sha256=4658c89c549a6e633742bbfb7a9633cd38e92f041e9eea975a4da522e61419e5
  Stored in directory: /root/.cache/pip/wheels/cc/fa/8c/16c93fccce688ae1bde7d979ff102f7bee980d9cfeb8641bcf
Successfully built scikit-surprise


In [ ]:
import surprise

print(surprise.__version__)

1.1.0


### Surprise를 이용한 추천 시스템 구축

In [ ]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

In [ ]:
data = Dataset.load_builtin('ml-100k')
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] 
Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


In [ ]:
algo = SVD()
algo.fit(trainset)

In [ ]:
predictions = algo.test( testset )
print('prediction type :',type(predictions), ' size:',len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

prediction type : <class 'list'>  size: 25000
prediction 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.31287014243118, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.7870793111066496, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=3.8952757349713454, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.613100757380748, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.3424063126966943, details={'was_impossible': False})]

In [ ]:
[ (pred.uid, pred.iid, pred.est) for pred in predictions[:3] ]

[('120', '282', 3.31287014243118),
 ('882', '291', 3.7870793111066496),
 ('535', '507', 3.8952757349713454)]

In [ ]:
# 사용자 아이디, 아이템 아이디는 문자열로 입력해야 함. 
uid = str(196)
iid = str(302)
pred = algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.16   {'was_impossible': False}


In [ ]:
accuracy.rmse(predictions)

RMSE: 0.9510


0.9510479354271372

### Surprise 주요 모듈 소개

In [ ]:
import pandas as pd
import numpy as np

import warnings; 
warnings.filterwarnings('ignore')

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
from surprise import Reader

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
data = Dataset.load_from_file('/content/gdrive/My Drive/notebook/ml-guide/data/ratings_noh.csv', reader=reader)

In [ ]:
 trainset, testset = train_test_split(data, test_size=.25, random_state=0)

 # 수행 시마다 동일한 결과를 도출하기 위해 random_state 설정
 algo = SVD(n_factors=50, random_state=0)

 # 학습 데이터 세트로 학습하고 나서 테스트 데이터 세트로 평점 예측 후 RMSE 평가
 algo.fit(trainset)
 predictions = algo.test(testset)
 accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

### 판다스 Dataframe에서 Surprise 데이터 세트로 로딩

In [ ]:
import pandas as pd
from surprise import Reader, Dataset


ratings = pd.read_csv('/content/gdrive/My Drive/notebook/ml-guide/data/ratings.csv')
reader = Reader(rating_scale=(0.5, 5.0))

data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)


RMSE: 0.8682


0.8681952927143516

### 교차 검증과 하이퍼 파라미터 튜닝

In [ ]:
from surprise.model_selection import cross_validate

ratings = pd.read_csv('/content/gdrive/My Drive/notebook/ml-guide/data/ratings.csv')
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)


algo = SVD(n_factors=50, random_state=0)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8692  0.8758  0.8567  0.8724  0.8720  0.8692  0.0066  
MAE (testset)     0.6695  0.6723  0.6608  0.6692  0.6697  0.6683  0.0039  
Fit time          3.47    3.44    3.49    3.44    3.47    3.46    0.02    
Test time         0.15    0.15    0.30    0.15    0.30    0.21    0.07    


{'fit_time': (3.4706411361694336,
  3.4399361610412598,
  3.491745948791504,
  3.437457799911499,
  3.4743692874908447),
 'test_mae': array([0.66951988, 0.67226949, 0.66082468, 0.66922065, 0.66973693]),
 'test_rmse': array([0.86921469, 0.87576137, 0.85666284, 0.87243289, 0.87199476]),
 'test_time': (0.14833641052246094,
  0.1495974063873291,
  0.2997415065765381,
  0.15413808822631836,
  0.30237793922424316)}

In [ ]:
from surprise.model_selection import GridSearchCV

# 최적화할 파라미터를 딕셔너리 형태로 지정.
param_grid = {'n_epochs': [20, 40, 60], 'n_factors': [50, 100, 200]}

# CV를 3개 폴드 세트로 지정, 성능 평가는 rmse, mse로 수행하도록 GridSearchCV 구성
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

# 최고 RMSE Evaluation 점수와 그때의 하이퍼 파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8775549652214565
{'n_epochs': 20, 'n_factors': 50}


### Surprise를 이용한 개인화 영화 추천 시스템 구축

In [ ]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
data_folds = DatasetAutoFolds(ratings_file='/content/gdrive/My Drive/notebook/ml-guide/data/ratings_noh.csv', reader=reader)

trainset = data_folds.build_full_trainset()

In [ ]:
movies = pd.read_csv('/content/gdrive/My Drive/notebook/ml-guide/data/movies.csv')
movieIds = ratings[ratings['userId'] == 9]['movieId']

if movieIds[movieIds==42].count() == 0:
  print('사용자 아이디 9는 영화 아이디 42의 평점 없음')

print(movies[movies['movieId'] == 42])  

사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [ ]:
uid = str(9)
iid = str(42)

pred = algo.predict(uid, iid, verbose=True)

user: 9          item: 42         r_ui = None   est = 3.50   {'was_impossible': False}


In [ ]:
def get_unseen_surprise(ratings, movies, userId):
  seen_movies = ratings[ratings['userId'] == userId]['movieId'].tolist()

  total_movies = movies['movieId'].tolist()

  unseen_movies = [movie for movie in total_movies if movie not in seen_movies]
  print('평점 매긴 영화 수:', len(seen_movies), '추천 대상 영화 수:', len(unseen_movies), '전체 영화 수:', len(total_movies))

  return unseen_movies

unseen_movies = get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화 수: 46 추천 대상 영화 수: 9696 전체 영화 수: 9742


In [ ]:
def recomm_movie_by_surpirse(algo, userId, unseen_movies, top_n=10):
  # 알고리즘 객체의 predict() 메서드를 평점이 없는 영화에 반복 수행한 후 결과를 list 객체로 저장
  predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]

  # predictions list 객체는 surprise의 Predictions 객체를 원소로 가지고 있음
  # [Prediction(uid='9', iid='1', est=3.69), Predictions(uid='9', iid='2', est=2.98),,,,]
  # 이를 est 값으로 정렬하기 위해서 아래의 sortkey_est 함수를 정의함.
  # sortkey_est 함수는 list 객체의 sort() 함수의 키 값으로 사용되어 정렬 수행.
  def sortkey_est(pred):
    return pred.est

  # sortkey_est() 반환값의 내림 차순으로 정렬 수행하고 top_n개의 최상위 값 추출.
  predictions.sort(key=sortkey_est, reverse=True)
  top_predictions = predictions[:top_n]

  # top_n으로 추출된 영화의 정보 추출. 영화 아이디, 추천 예상 평점, 제목 추출
  top_movie_ids = [ int(pred.iid) for pred in top_predictions ]
  top_movie_rating = [ pred.est for pred in top_predictions ]
  top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']

  top_movie_preds = [ (id, title, rating) for id, title, rating in zip(top_movie_ids, top_movie_titles, top_movie_rating)]

  return top_movie_preds

unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surpirse(algo, 9, unseen_movies, top_n=10)

print(' ##### TOP 10 추천 영화 리스트 ##### ')
for top_movie in top_movie_preds:
  print(top_movie[1], ":", top_movie[2])


평점 매긴 영화 수: 46 추천 대상 영화 수: 9696 전체 영화 수: 9742
 ##### TOP 10 추천 영화 리스트 ##### 
Toy Story (1995) : 3.503353208791481
Jumanji (1995) : 3.503353208791481
Grumpier Old Men (1995) : 3.503353208791481
Waiting to Exhale (1995) : 3.503353208791481
Father of the Bride Part II (1995) : 3.503353208791481
Heat (1995) : 3.503353208791481
Sabrina (1995) : 3.503353208791481
Tom and Huck (1995) : 3.503353208791481
Sudden Death (1995) : 3.503353208791481
GoldenEye (1995) : 3.503353208791481
